### **Practise I**

**API for Systems Diagnostics**
We will create a simple API for remote diagnostics of a planetary base's systems. We will prepare a `GET` endpoint that returns data and a `POST` endpoint that accepts a command and returns a custom response.

**Preparation:**
For this practise, use the following file as a base. Your task is to complete the code in the indicated places.
```python
from flask import Flask, jsonify, make_response, request
import uuid # generates a unique ID

app = Flask(__name__)

# --- Data Source ---
# In a real app, this would come from a database or live sensors.
SYSTEM_STATUSES = [
    {
        "system": "Life Support",
        "status": "Nominal",
        "details": "O2 Levels at 98%"
    },
    {
        "system": "Primary Power",
        "status": "Warning",
        "details": "Fluctuation detected in reactor core"
    },
    {
        "system": "Shield Generator",
        "status": "Active",
        "details": "Power draw at 75%"
    },
    {
        "system": "Communications Array",
        "status": "Offline",
        "details": "Signal lost with Orbiter One. Awaiting realignment."
    },
    {
        "system": "Drilling Laser",
        "status": "Standby",
        "details": "Awaiting command to initiate sequence."
    }
]

# --- API Endpoints ---

# Create the /api/status ENDPOINT here

# Create the /api/run_diagnostic ENDPOINT here


# --- App ---
if __name__ == "__main__":
    app.run(debug=True)
```

**Assignment:**

1.  **Create a `GET` endpoint for reading status:**
    * Build a new route `/api/status` that accepts `GET` requests.
    * It should return the `SYSTEM_STATUSES` dictionary in JSON format and add a status code.

2.  **Create a `POST` endpoint to run diagnostics:**
    * Build a new route `/api/run_diagnostic` that accepts `POST` requests.
    * It will simulate starting a long-running task.
    * Create a JSON response with a confirmation, e.g., `{"message": "Diagnostic sequence initiated"}`.
    * Use `make_response()` to "wrap" this JSON response so you can:
        * Set the status code to `202` (request has been accepted for processing).
        * Add a custom header `X-Request-ID`, whose value will be a unique ID (you can use `str(uuid.uuid4())`).

In [ ]:
from flask import Flask, jsonify, make_response
import uuid # generates a unique ID

app = Flask(__name__)

# --- Data Source ---
# In a real app, this would come from a database or live sensors.
SYSTEM_STATUSES = [
    {
        "system": "Life Support",
        "status": "Nominal",
        "details": "O2 Levels at 98%"
    },
    {
        "system": "Primary Power",
        "status": "Warning",
        "details": "Fluctuation detected in reactor core"
    },
    {
        "system": "Shield Generator",
        "status": "Active",
        "details": "Power draw at 75%"
    },
    {
        "system": "Communications Array",
        "status": "Offline",
        "details": "Signal lost with Orbiter One. Awaiting realignment."
    },
    {
        "system": "Drilling Laser",
        "status": "Standby",
        "details": "Awaiting command to initiate sequence."
    }
]

# --- API Endpoints ---

@app.route("/api/status", methods=["GET"])
def get_system_status():
    """
    Returns the current status of all systems.
    """
    return jsonify(SYSTEM_STATUSES), 200


@app.route("/api/run_diagnostic", methods=["POST"])
def run_diagnostic():
    """
    Simulates initiating a long-running diagnostic task.
    """
    # Generate a unique ID
    request_id = str(uuid.uuid4()) 

    # Create the base JSON payload
    json_payload = jsonify({
        "message": "Diagnostic sequence initiated.",
        "request_id": request_id
    })

    # Use make_response to wrap the jsonify response and add custom data.
    response = make_response(json_payload, 202)

    # Add a custom header
    response.headers['X-Request-ID'] = request_id

    return response


# --- App ---
if __name__ == "__main__":
    app.run(debug=True)

### **Practise II**
**Automated Diagnostic Script**

We will create an automated bot (an external Python script) that will connect to our running application's API, download the system status data, and automatically check for warnings. We will utilize the capabilities and data from our API.

**Preparation:**
1.  In one terminal, **keep the Flask application from the previous practise running**.
2.  Create a **new, separate file** `diagnostic_check.py`.
3.  Install the `requests` library.
    ```bash
    pip install requests
    ```

**Assignment:**
1.  In the `diagnostic_check.py` file, import the `requests` library.
2.  Define a variable with the URL of your local API endpoint: `http://127.0.0.1:5000/api/status`.
3.  Using `requests.get()`, send a `GET` request to this address.
4.  From the response object, get the data in JSON format (using the `.json()` method).
5.  Using a loop, iterate through the received data.
6.  If you encounter a `status` with the value `"Warning"` for any system, print a warning message to the console, for example:
    `WARNING: System 'Primary Power' reports a warning! Details: 'Fluctuation detected in reactor core'`

In [ ]:
import requests

# The URL of our local Flask API endpoint
API_URL = "http://127.0.0.1:5000/api/status"

try:

    response = requests.get(API_URL)
    response.raise_for_status()

    # Parse the JSON response into a Python list of dictionaries
    all_statuses = response.json()

# --- Searching & analyzing data ---
    print("Connection successful. Analyzing system data...")
    found_warning = False

    for system in all_statuses:
        if system.get("status") == "Warning":
            print(f"\n!!! WARNING DETECTED !!!")
            print(f"System: {system.get('system')}")
            print(f"Details: {system.get('details')}")
            found_warning = True
    
    if not found_warning:
        print("\nAnalysis complete. All systems nominal.")

except requests.exceptions.RequestException as e:
    print(f"\nERROR: Could not connect to the API.")
    print(f"Please ensure the Flask application from Practise I is running in another terminal.")
    print(f"Details: {e}")

### **Project (Homework): Upgrade to an Automated Data Channel**

**Mission:** Our web-based information system is great for the crew, but autonomous systems and Mission Control on Earth need access to raw, real-time data, not formatted HTML pages. Our task is to create two new API endpoints that will serve as a standardized data channel for machine-to-machine communication.

`Basic application structure`:
- my_project/
  - my_app.py
  - routes.py
  - forms.py
  - database.py
  - models.py
  - **data/**
    - **asteroid.py**
    - **crew.py**
  - templates/
  - static/
  - config/

- We will keep our **SQLite DB** file in the default **instance** folder outside our application.

**1. Data Channel for the Information System:**
* Create a new API route `/api/crew_members` that will accept `GET` requests.
* Query the database to get **all** records of crew members.
* Convert them to a **list of dictionaries** and pass it to `jsonify()`.
* Return it as a JSON response with a `200 OK` status code.

**2. Telemetry of the Target Asteroid:**
* Create another API route `/api/target_asteroid` that will also accept `GET` requests.
* Return the data about `my_asteroid` as a JSON response using `jsonify()`.

**3. Data Stream Upgrade:**
* Move the data about the crew and the asteroid from the application file to separate files `asteroid.py` and `crew.py` in a new `data` folder - we will import them from there into our application.

**4. Transmission Verification:**
* After starting the application, verify the functionality of both new endpoints = enter them into a web browser. You should see clean, raw data in JSON format.

In [ ]:
# configuration.py

class Config:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True 
    STATIC_FOLDER = 'static' 
    TEMPLATES_FOLDER = 'templates' 
    SECRET_KEY = 'my_secret_something' 

    # Database Config (set up DB connection) - for SQLite
    SQLALCHEMY_DATABASE_URI = 'sqlite:///my_database.db' 
    SQLALCHEMY_TRACK_MODIFICATIONS = False 

In [ ]:
from flask import Flask
from config.configuration import Config 
from routes import register_routes
from database import db
from models import CrewMember
from data.asteroid import my_asteroid
from data.crew import my_crew


app = Flask(__name__)


# -------------- CONFIGURATION --------------
app.config.from_object(Config)


# -------------- INITIALIZATION --------------
db.init_app(app)
register_routes(app, db, my_asteroid)


# -------------- TABLEs - creation --------------
with app.app_context(): 
    db.create_all() # creates tables (if no tables in DB)
    if not CrewMember.query.first(): # populates database (if DB is empty)
        for crew_member in my_crew:
            new_crew_member = CrewMember(
                id=crew_member["id"], 
                name=crew_member["name"], 
                specialization=crew_member["specialization"], 
                status=crew_member["status"])
            db.session.add(new_crew_member)
        db.session.commit()


# -------------- STARTING THE APP --------------
if __name__ == "__main__":
    app.run()

In [ ]:
# routes.py

from flask import render_template, redirect, url_for, jsonify
from forms import NewCrewMemberForm 
from models import CrewMember


def register_routes(app, db, my_asteroid):
    @app.route("/")
    def homepage():
        return render_template("homepage.html")

    @app.route("/mission_briefing")
    def mission_briefing():
        return render_template("mission_briefing.html")

    @app.route("/target_asteroid")
    def target_asteroid():
        return render_template("target_asteroid.html", my_asteroid=my_asteroid)

    @app.route("/crew_members")
    def crew_members():
        my_crew = db.session.query(CrewMember).all()
        return render_template("crew_members.html", my_crew=my_crew)

    @app.route("/crew_members/<int:id>")
    def crew_members_details(id):
        crew_member = db.session.query(CrewMember).get(id)
        if crew_member:
            return render_template("crew_members_details.html", crew_member=crew_member)
        return "Crew member not found"
    
    @app.route("/onboarding", methods=["GET", "POST"])
    def onboarding():
        form = NewCrewMemberForm()
        if form.validate_on_submit():
            db.session.add(CrewMember(
                name=form.name.data,
                specialization=form.specialization.data,
                status="Pending.."
            ))
            db.session.commit()
            return redirect(url_for("crew_members"))
        return render_template("onboarding.html", form=form)
    
    
# -------------- API - ENDPOINTS --------------
    @app.route("/api/crew_members")
    def api_crew_members():
        all_crew_members = db.session.query(CrewMember).all()
        return jsonify(list({"id":member.id, "name":member.name, "specialization":member.specialization, "status":member.status} for member in all_crew_members)), 200
    
    @app.route("/api/target_asteroid")
    def api_target_asteroid():
        return jsonify(my_asteroid), 200

In [ ]:
# forms.py

from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, validators

class NewCrewMemberForm(FlaskForm):
    name = StringField(label="Name", validators=[validators.DataRequired()])
    specialization = StringField(label="Specialization", validators=[validators.DataRequired()])
    
    submit = SubmitField(label="Submit new crew member")

In [ ]:
# database.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

In [ ]:
# models.py

from database import db 


class CrewMember(db.Model):
    __tablename__ = "crew_members"
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    specialization = db.Column(db.String(100), nullable=False)
    status = db.Column(db.String(100), nullable=False)

    def __repr__(self):
        return f"<CrewMember {self.name}>"

## /data

In [ ]:
# data/asteroid

my_asteroid = {"name": "230 Athamantis",
               "diameter": "118±2 km",
               "mass": "(2.3±1.1)*10**18 kg",
               "density": "2.7±1.3 g/cm3",
               "speed": "19.3 km/s",
               "img": "static/img/Athamantis.jpeg"
}

In [ ]:
# data/crew

my_crew = [
        {
            "id": 1,
            "name": "Dave Fisher",
            "specialization": "AI Engineer",
            "status": "Active"
        },
        {
            "id": 2,
            "name": "Izael Alexander",
            "specialization": "Rocket Scientist",
            "status": "Active"
        },
        {
            "id": 3,
            "name": "Tethra Dyagran",
            "specialization": "Roboticist",
            "status": "Active"
        },
        {
            "id": 4,
            "name": "Mura Lan",
            "specialization": "Astronavigator",
            "status": "Active"
        }
    ]

## /templates

In [ ]:
<!-- header.html -->
 
<nav>
    <ul>
        <li><a href="{{ url_for('homepage') }}">Home</a></li> 
        <li><a href="{{ url_for('mission_briefing') }}">Our Mission</a></li>
        <li><a href="{{ url_for('target_asteroid') }}">The Asteroid</a></li>
        <li><a href="{{ url_for('crew_members') }}">Our Crew</a></li>
        <li><a href="{{ url_for('onboarding') }}">New Recruits Onboarding</a></li>
    </ul>
</nav>

In [ ]:
<!-- footer.html -->
 
<footer>
    <br>
    <p>Deep Space Exploration Corp.</p>
    <p>All rights reserved</p>
</footer>

In [ ]:
<!-- base.html -->

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block page_title %} Title {% endblock page_title %}</title>
    
</head>

<body>
    {% include 'header.html' %} 
    {% block page_content %} Content {% endblock page_content %} 
    {% include 'footer.html' %} 
</body>

</html>

In [ ]:
<!-- homepage.html -->

{% extends "base.html" %} 

{% block page_title %} HomePage {% endblock page_title %}

{% block page_content %}
    <h1>Deep Space Exploration Corp.</h1>
    <p>Welcome to our homepage !</p>
    <p>Our goal is to mine asteroids from the solar system.</p>
{% endblock page_content %}

In [ ]:
<!-- mission_briefing.html -->

{% extends "base.html" %} 

{% block page_title %} Mission Debriefing {% endblock page_title %}

{% block page_content %}
    <h1>Mission</h1>
    <p>This is our mission:</p>
    <p>get resources from chosen asteroids</p>
    <p>make it profitable</p>
    <p>expand world's economy</p>
{% endblock page_content %}

In [ ]:
<!-- target_asteroid.html -->

{% extends "base.html" %} 

{% block page_title %} Target Asteroid {% endblock page_title %}

{% block page_content %}
    <h1>Target asteroid: {{ my_asteroid["name"] }}</h1>
    <p>Mean diameter: {{ my_asteroid["diameter"] }}</p>
    <p>Mass: {{ my_asteroid["mass"] }}</p>
    <p>Mean density: {{ my_asteroid["density"] }}</p>
    <p>Average orbital speed: {{ my_asteroid["speed"] }}</p>
    <img src="{{ my_asteroid['img'] }}" width="800px"/>  
{% endblock page_content %}

In [ ]:
<!-- crew_members_details.html -->

{% extends "base.html" %} 

{% block page_title %} {{ crew_member["name"] }} {% endblock page_title %}

{% block page_content %}
    <h1>Crew member {{ crew_member["id"] }}</h1>
    <p>Name: {{ crew_member["name"] }}</p>
    <p>Specialization: {{ crew_member["specialization"] }}</p>
    <p>Status: {{ crew_member["status"] }}</p>
{% endblock page_content %}

In [ ]:
<!-- crew_members.html -->

{% extends "base.html" %} 

{% block page_title %} Our Crew {% endblock page_title %}

{% block page_content %}
    {% for member in my_crew %}
        <p><a href="{{ url_for('crew_members_details', id=member['id']) }}">{{ member["name"] }}</a></p>
    {% endfor %}
{% endblock page_content %}

In [ ]:
<!-- onboarding.html -->

{% extends "base.html" %} 

{% block page_title %} Onboarding {% endblock page_title %}

{% block page_content %}
<form method = "POST" action = "{{ url_for('onboarding') }}">
    <p>New crew member onboarding form</p>
    {{ form.hidden_tag() }}

    <p>{{ form.name.label }} {{ form.name(size=30) }}</p>
    <p>{{ form.specialization.label }} {{ form.specialization(size=30) }}</p>

    <p>{{ form.submit() }}</p>
</form>
{% endblock page_content %}

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom